<a href="https://colab.research.google.com/github/ameympatil/MIT-ML/blob/main/keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
import numpy as np
import pandas as pd

In [100]:
df = pd.read_csv('/content/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [101]:
df.shape

(768, 9)

In [102]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [103]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [104]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [105]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [106]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

## Without tuning

In [107]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [108]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 1s 10ms/step - loss: 0.6196 - accuracy: 0.6808 - val_loss: 0.6324 - val_accuracy: 0.6494
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5743 - accuracy: 0.7199 - val_loss: 0.5891 - val_accuracy: 0.7078
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5430 - accuracy: 0.7427 - val_loss: 0.5603 - val_accuracy: 0.7273
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5219 - accuracy: 0.7492 - val_loss: 0.5373 - val_accuracy: 0.7078
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5069 - accuracy: 0.7622 - val_loss: 0.5220 - val_accuracy: 0.7532
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4969 - accuracy: 0.7622 - val_loss: 0.5081 - val_accuracy: 0.7597
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4879 - accuracy: 0.7687 - val_loss: 0.5010 - val_accuracy: 0.7727
Epoch 8/100


# How to select appropriate optimizer

In [109]:
#pip install -U keras-tuner

In [110]:
import kerastuner as kt

In [113]:
def build_model(hp):
  model = Sequential()


  units = hp.Int('units',min_value=8,max_value=128,step=8)

  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [114]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

In [115]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [117]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam', 'units': 8}

In [121]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 32)                288       
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________
